In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
!pip install scikit-learn

In [3]:
df = pd.read_csv("c:/Users/pipil/OneDrive/Работен плот/LanguageAI/LanguageAndAI/Data/birth_year.csv")
df.dropna(subset=['auhtor_ID', 'post', 'birth_year'], inplace=True)
df.head(5)

,auhtor_ID,post,birth_year
0,t2_rnjzutp,Good on you for being responsible! I know self...,2001
1,t2_rnjzutp,"must go to the grocery store with their child,...",2001
2,t2_rnjzutp,"things on her videos, and YouTube took the vid...",2001
3,t2_rnjzutp,their app. There's also a program called SYNC ...,2001
4,t2_rnjzutp,"side. If the cops don't take your side, you'll...",2001


In [4]:
#TF-IDF Feature extraction
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # We can tune max_features later on 
X = tfidf_vectorizer.fit_transform(df['post'])

In [5]:
y = df['birth_year']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
regressor = DecisionTreeRegressor()
regressor.fit(X_train, y_train)


DecisionTreeRegressor()

In [18]:
y_pred = regressor.predict(X_test)
#MSE for Decision Tree
mse_dt= mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse_dt}')

Mean Squared Error: 156.81480597014925


In [10]:
# Implementing Random forest on the ssame split
rf_regressor = RandomForestRegressor(n_estimators=15, random_state=42) #n_estimators=15 took around 4.30 hours to run, n_estimators=4 took 15 minutes. 
rf_regressor.fit(X_train, y_train)



RandomForestRegressor(n_estimators=15, random_state=42)

In [19]:
y_pred_rf = rf_regressor.predict(X_test)
#MSE for Random Forest 
mse_rf = mean_squared_error(y_test, y_pred_rf)
print(f'Mean Squared Error: {mse_rf}')


Mean Squared Error: 81.29480543946933


In [ ]:
#These are the two models in a class. I am not sure if this is the right structure and if it is 
#gonna fit with the Evaluation class but I made it like that as a starting point.
# I am not sure if it works tho the whole thing is probably gonna take around 5 hours to run 

class TextRegressionModel:
    def __init__(self, model_type='random_forest', max_features=1000, n_estimators=15, random_state=42):
        self.vectorizer = TfidfVectorizer(max_features=max_features)
        if model_type == 'random_forest':
            self.model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
        elif model_type == 'decision_tree':
            self.model = DecisionTreeRegressor(random_state=random_state)
        else:
            raise ValueError("Model type not supported. Choose 'random_forest' or 'decision_tree'.")

    def preprocess_data(self, df):
        X = self.vectorizer.fit_transform(df['post'])
        y = df['birth_year']
        return X, y

    def train_model(self, X_train, y_train):
        self.model.fit(X_train, y_train)

    def make_predictions(self, X):
        return self.model.predict(X)

if __name__ == "__main__":
    
    # Random Forest Model
    text_model_rf = TextRegressionModel(model_type='random_forest')
    X, y = text_model_rf.preprocess_data(df)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    text_model_rf.train_model(X_train, y_train)
    y_pred_rf = text_model_rf.make_predictions(X_test)

    # Decision Tree Model
    text_model_dt = TextRegressionModel(model_type='decision_tree')
    # Reusing the same preprocessed data
    text_model_dt.train_model(X_train, y_train)
    y_pred_dt = text_model_dt.make_predictions(X_test)